In [1]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

with open('stop_words_en.txt', "r") as f:
    stop_words = set(f.read().splitlines())
    
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    text= re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    text = re.split("\W*\s+\W*", text, flags=re.UNICODE)   
    
    counter = dict()
    for w in text:
        w = w.lower()
        if w in stop_words:
            continue
            
        if w in counter:
            counter[w] += 1
        else:
            counter[w] = 1
    
    total = sum(counter.values())
    for term, cnt in counter.iteritems():
        print "%s\t%s\t%f" % (term, article_id, float(cnt)/total)

Writing mapper1.py


In [4]:
%%writefile reducer.py

import sys
from math import log
from imp import reload
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8") # required to convert to unicode

articles_dict = dict()
current_key = None

for line in sys.stdin:
    try:
        key, article_id, tf = line.strip().split('\t', 2)
        tf = float(tf)

        if current_key != key:
            if current_key:
                idf = (float(1)/log(1 + len(articles_dict.values())))
                for key_article_id, tf in articles_dict.items():
                    tf_idf = tf * idf
                    print ("%s\t%s\t%f" % (current_key, key_article_id, tf_idf))

            current_key = key
            articles_dict = dict()

        articles_dict[article_id] = float(tf)
    
    except Exception as e:
        print("Error in reducer.py", e)
        continue
    
if current_key:
    idf = (float(1)/log(1 + len(articles_dict.values())))
    for key_article_id, tf in articles_dict.items():
        tf_idf = tf * idf
        print ("%s\t%s\t%f" % (current_key, key_article_id, tf_idf))

Writing reducer.py


In [3]:
%%bash

OUT_DIR="out_"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming TF-IDF" \
    -D mapreduce.job.reduces=4 \
    -files mapper1.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python2 mapper1.py" \
    -reducer "python3 reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/* |  grep -P 'labor\t12\t' | cut -f3
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

Couldn't find program: 'bash'
